## Task 1

In [4]:
import sqlite3

In [5]:
conn = sqlite3.connect("sensor_readings.db")

In [6]:
c = conn.cursor()

#1.a)

c.execute("SELECT name FROM sqlite_master")
results = c.fetchall()

print([table[0] for table in results])

['measurements', 'sensors']


In [7]:
#1.b)

c.execute("SELECT * FROM sensors")
results = c.fetchall()

print(c.description)
print(results)

(('sensorid', None, None, None, None, None, None), ('sensor_type', None, None, None, None, None, None), ('channel', None, None, None, None, None, None))
[(0.0, 'Accelerometer', 'x'), (1.0, 'Accelerometer', 'y'), (2.0, 'Accelerometer', 'z'), (3.0, 'Gyroscope', 'x'), (4.0, 'Gyroscope', 'y'), (5.0, 'Gyroscope', 'z'), (6.0, 'Magnetometer', 'x'), (7.0, 'Magnetometer', 'y'), (8.0, 'Magnetometer', 'z'), (9.0, 'Gravity', 'x'), (10.0, 'Gravity', 'y'), (11.0, 'LinearAcceleration', 'x'), (12.0, 'LinearAcceleration', 'y'), (13.0, 'Gravity', 'z'), (14.0, 'RotationVector', 'x'), (15.0, 'LinearAcceleration', 'z'), (16.0, 'RotationVector', 'y'), (17.0, 'RotationVector', 'z'), (18.0, 'noise', 'decibels')]


In [8]:
import pandas as pd
df = pd.read_sql_query("SELECT * FROM sensors;", conn)
print(df)

    sensorid         sensor_type   channel
0        0.0       Accelerometer         x
1        1.0       Accelerometer         y
2        2.0       Accelerometer         z
3        3.0           Gyroscope         x
4        4.0           Gyroscope         y
5        5.0           Gyroscope         z
6        6.0        Magnetometer         x
7        7.0        Magnetometer         y
8        8.0        Magnetometer         z
9        9.0             Gravity         x
10      10.0             Gravity         y
11      11.0  LinearAcceleration         x
12      12.0  LinearAcceleration         y
13      13.0             Gravity         z
14      14.0      RotationVector         x
15      15.0  LinearAcceleration         z
16      16.0      RotationVector         y
17      17.0      RotationVector         z
18      18.0               noise  decibels


In [9]:
#1.c)

#1.b)

c.execute("SELECT * FROM measurements;")
results = c.fetchall()

df = pd.read_sql_query("SELECT * FROM measurements;", conn)
print(df)


       sensorid     timestamp      data
0           0.0  1.556136e+09 -0.605637
1           1.0  1.556136e+09  1.623474
2           2.0  1.556136e+09  9.710114
3           3.0  1.556136e+09  0.002228
4           4.0  1.556136e+09  0.005508
...         ...           ...       ...
23068       0.0  1.556137e+09 -0.816330
23069       1.0  1.556137e+09  0.969849
23070       3.0  1.556137e+09  0.013412
23071       2.0  1.556137e+09  9.927994
23072       4.0  1.556137e+09  0.065582

[23073 rows x 3 columns]


In [10]:
#1.c)
query = """
SELECT m.sensorid, s.sensor_type, count(*), min(m.data), max(m.data)
FROM measurements AS m
INNER JOIN sensors AS s ON m.sensorid = s.sensorid
GROUP BY m.sensorid
"""
c.execute(query)
c.fetchall()
df = pd.read_sql_query(query, conn)
print(df)

    sensorid         sensor_type  count(*)  min(m.data)  max(m.data)
0        0.0       Accelerometer      1349    -6.830551    15.428329
1        1.0       Accelerometer      1340    -9.574219     8.530746
2        2.0       Accelerometer      1334   -19.585342    56.121720
3        3.0           Gyroscope      1334    -8.439453     3.792435
4        4.0           Gyroscope      1324   -16.223800    14.743439
5        5.0           Gyroscope      1313    -6.092300     4.696701
6        6.0        Magnetometer      1307   -58.802795    49.629210
7        7.0        Magnetometer      1302   -46.693420    41.267395
8        8.0        Magnetometer      1287   -50.630188    51.737976
9        9.0             Gravity      1282    -9.568751     9.805525
10      10.0             Gravity      1265    -9.748381     9.782932
11      11.0  LinearAcceleration      1175    -7.031983     6.909350
12      12.0  LinearAcceleration      1139   -11.810923    13.980898
13      13.0             Gravity  

In [11]:
#2.

In [23]:
#2.a)
from tinydb import TinyDB, Query, where
db = TinyDB('sensor_reports.json')
doc = Query()

print(db.all()[-2:])

[{'type': 'sensor', 'sensor_type': 'RotationVector', 'channel': 'z'}, {'type': 'sensor', 'sensor_type': 'noise', 'channel': 'decibels'}]


In [24]:
from pprint import pprint
pprint(db.all()[0])

{'state': 'anomaly',
 'timestamp': 1556136466.2272274,
 'topic': 'aidae/1/noise/decibels',
 'type': 'report',
 'value': '-inf'}


In [28]:
#2.c)
db.search(doc.state == 'anomaly')

[{'type': 'report',
  'state': 'anomaly',
  'value': '-inf',
  'topic': 'aidae/1/noise/decibels',
  'timestamp': 1556136466.2272274}]

In [32]:
#3
##3.a)
import numpy as np

class SinSensor:
    def __init__(self, amplitude, variance):
        self.amplitude = amplitude
        self.variance = variance
    def getMeasurements(self, t):
        return np.sin(t) *self.amplitude + np.random.normal(0, self.variance)
    
sensor = SinSensor(5,0.2)
        

In [33]:
sensor.getMeasurements(0.1)

0.22126737597620827

In [43]:
##3.b)

import paho.mqtt.client as mqtt
import time

broker = "broker.hivemq.com"
topic = "aidae/12345/SinSensor"
interval = 0.2

client = mqtt.Client()
client.connect(broker)

t = 0
while True:
    message = str(sensor.getMeasurements(t * interval))
    client.publish(topic, message)
    print("Published Data", message)
    t += 1
    time.sleep(interval)

Published Data 0.3134481826080475
Published Data 1.0056171409165564
Published Data 2.1010904977675837
Published Data 3.1111107410515375
Published Data 3.3611674408626566
Published Data 4.141229709961496
Published Data 4.5658600526129725
Published Data 4.848964866584924
Published Data 4.6014634729469766
Published Data 5.1491573109739575
Published Data 4.582850987508437
Published Data 3.9351583655699023
Published Data 3.6043809132072555
Published Data 2.531090044620612
Published Data 1.6269511526144702
Published Data 1.0618427571665574
Published Data -0.3491153135578222
Published Data -1.0010973236646121
Published Data -2.272784564515075
Published Data -2.881826307313546
Published Data -3.8845151539866993
Published Data -4.590435781030875
Published Data -4.606839832187525
Published Data -4.824137395335093
Published Data -5.589023893379713
Published Data -4.7478321100031
Published Data -4.176493878605331
Published Data -3.9255389477323
Published Data -3.1231969699607696
Published Data -2.

Published Data -4.966328676379329
Published Data -4.566057030386046
Published Data -4.477937419699132
Published Data -3.908997027448369
Published Data -3.297395052792965
Published Data -2.1601288661482463
Published Data -1.0572306936601337
Published Data -0.5092869613951158
Published Data 0.3349124507741685
Published Data 1.600809748884482
Published Data 2.7278027391884914
Published Data 3.583956632848073
Published Data 4.102687631188478
Published Data 4.812753204836379
Published Data 4.782269172101597
Published Data 4.9281127249309495
Published Data 4.850825613640078
Published Data 4.791991712455449
Published Data 4.330113314822027
Published Data 3.802244482726456
Published Data 3.2148981993014805
Published Data 2.0515436480801323
Published Data 0.9677623603606431
Published Data 0.16636511583402472
Published Data -0.7618010296485165
Published Data -2.0531811256359283
Published Data -2.8021664539636517
Published Data -3.7336296078109457
Published Data -4.554688875541199
Published Data 

Published Data 0.9498388432285273
Published Data 0.19085397834695117
Published Data -1.333659016194048
Published Data -1.944638331654229
Published Data -2.6563592391180504
Published Data -3.6596160858637083
Published Data -4.04628738642555
Published Data -4.702684076112284
Published Data -4.860766798912997
Published Data -4.938509544647296
Published Data -4.869375558786425
Published Data -4.569054746560422
Published Data -4.1572657736016545
Published Data -3.57504670381193
Published Data -2.5481196015962424
Published Data -2.1135350714401153
Published Data -0.4992368229591376
Published Data 0.11323992777428094
Published Data 1.4588572570866396
Published Data 2.546749736758087
Published Data 3.1412978739527824
Published Data 3.7232136900675057
Published Data 4.285464903142468
Published Data 4.903482739720978
Published Data 5.0013660405150935
Published Data 5.102779504511029
Published Data 4.7707330159309524
Published Data 4.23337616697495
Published Data 3.765063635826537
Published Data 

Published Data 4.464128556763245
Published Data 4.5509740501379286
Published Data 5.093665732977914
Published Data 4.921423871711361
Published Data 4.969027312563384
Published Data 4.326563752977549
Published Data 3.441924675220158
Published Data 3.3770913504037727
Published Data 2.2420674285709414
Published Data 1.3999930041846809
Published Data 0.22944679104968652
Published Data -0.7029299529226942
Published Data -1.9359662035288383
Published Data -2.9589598570437445
Published Data -2.9654917242049015
Published Data -4.058439421746223
Published Data -4.60351255183117
Published Data -4.752871147419168
Published Data -5.203225911153415
Published Data -4.864929824378268
Published Data -4.850535456035065
Published Data -3.913670552555853
Published Data -3.2611213292723735
Published Data -2.737288710522653
Published Data -1.5232113317809495
Published Data -1.2348301004634719
Published Data 0.1784303306973678
Published Data 1.2279810107877989
Published Data 1.8360869461460219
Published Dat

KeyboardInterrupt: 

In [ ]:
import paho.mqtt.client as mqtt
import time

broker = "broker.hivemq.com"
topic = "aidae/12345/SinSensor"
interval = 0.2

def on_connect(client, userdata, flags, rc):
    print("Yes, it got connected")
    client.subscribe(topic + "/#")
    
def on_message(client, userdata, msg):
    print("Msg Rcved", str(msg.payload.decode()))


client2 = mqtt.Client()
client2.on_connect(broker) = on_connect
client2.on_message = on_message

client2.loop_forever()